#### 贝叶斯拼写检查器
@train(features)
该函数是对语料库的的词频进行统计。\
model = collections.defaultdict(lambda:1):是对所有词都默认出现过一次。为了解决下面这个问题，要是遇到我们从来没有见过的新词怎么办？假如说有一个词拼写完全的正确，但是在语料库中没有包含这个词，从而这个词也永远不会出现在训练集中，从而这个词也永远不会出现在训练集中，于是，我们就要返回出现这个词的概率是0，这个情况不太妙。因为概率为0这个代表了这个时间绝对不可能发生，而在我们的概率模型中，我们期望用一个很小的概率来代表这种情况， lamba:1。 然后多出现一次就加1

In [43]:
import re, collections

# 把语料库中的单词全部抽取出来，转换成小写的，并且去除单词中间的特殊符号 
def words(text):
    return re.findall('[a-z]+', text.lower())   #re.findall() , 列出所有的匹配项，返回一个列表
 
# 对语料库的词频进行统计
def train(features):
    model = collections.defaultdict(lambda:1)
    for f in features:
        model[f] += 1
    return model

NWORDS = train(words(open('big.txt').read()))  #打开一个语料库
alphabet = 'abcdefghijklmnopqrstuvwxyz'

#### 编辑距离：
两个词之间的距离定义为使用了几次插入（在词中插入一个单字母），删除（删除一个单字母），交换（交换两个相邻字母），替换（把一个字母换成另一个）的操作从一个词编导另一个词

In [45]:
# 返回所有与单词 w 编辑距离为 1 的集合
def edits1(word):
    n = len(word)
    return set([word[0:i] + word[i+1:] for i in range(n)] + 
               [word[0:i] + word[i+1] +word[i] +word[i+2:] for i in range(n-1)] + 
               [word[0:i] + c + word[i+1:] for i in range(n) for c in alphabet] +
               [word[0:i] + c + word[i:] for i in range(n+1) for c in alphabet])  

In [46]:
#返回所有与单词 w 编辑距离为 2 的集合
#在这些编辑距离小于 2 的词中间，只把那些正确的词作为候选词
def known_edits2(word):
    return set(e2 for e1 in edits1(word) for e2 in edits1(e1) if e2 in NWORDS)

def known(word):
    return set(w for w in word if w in NWORDS)

#### 矫正方法
为了简单起便：我们这里认为编辑距离为1的正确单词比编辑距离为2的优先级高，而编辑距离为0的正确单词优先级比编辑距离为1的高

In [47]:
def correct(word):
    candidates = known([word]) or known(edits1(word)) or known_edits2(word) or [word]
    return max(candidates , key= lambda w:NWORDS[w])

In [48]:
correct("morn")

'more'